In [15]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv("Resources/tmdb_5000_movies.csv")
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
print(df.columns)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [5]:
def unpack_col(dict_list):
    print(dict_list)
    for d in dict_list:
        print(d)
        print(d['name'])
    return {d['name']:1 for d in dict_list}


In [6]:
genres_dummies = df['genres'].apply(json.loads).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).str.join('|').str.get_dummies()
genres_dummies.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [7]:
keywords_dummies = df['keywords'].apply(json.loads).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).str.join('|').str.get_dummies()
keywords_dummies.head()

,15th century,16th century,17th century,18th century,1910s,1920s,1930s,1940s,1950s,1960s,...,flipping coin,gilbert and sullivan,nightgown,north carolinam,nosferatu,strange noise,Γη,卧底肥妈,绝地奶霸,超级妈妈
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
prod_companies_dummies = df['production_companies'].apply(json.loads).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).str.join('|').str.get_dummies()
prod_companies_dummies.head()

,"""DIA"" Productions GmbH & Co. KG",1.85 Films,10 West Studios,100 Bares,1019 Entertainment,101st Street Films,10th Hole Productions,120 Films,120dB Films,13 Ghosts Productions Canada Inc.,...,rusty bear entertainment,thefyzz,thinkfilm,uFilm,unafilm,verture Films,warner bross Turkey,winchester films,África Filmes,Österreichischer Rundfunk (ORF)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
prod_countries_dummies = df['production_countries'].apply(json.loads).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).str.join('|').str.get_dummies()
prod_countries_dummies.head()

,Afghanistan,Algeria,Angola,Argentina,Aruba,Australia,Austria,Bahamas,Belgium,Bhutan,...,Sweden,Switzerland,Taiwan,Thailand,Tunisia,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States of America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
spoken_lang_dummies = df['spoken_languages'].apply(json.loads).apply(lambda x: [i['iso_639_1'] for i in x] if isinstance(x, list) else []).str.join('|').str.get_dummies()
spoken_lang_dummies.head()

,af,am,ar,bg,bm,bn,bo,br,bs,ca,...,tr,uk,ur,vi,wo,xh,xx,yi,zh,zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Drop columns
columns_to_drop = ['original_title', 'overview', 'tagline', 'homepage', 'original_language', 'status']
df.drop(columns=columns_to_drop, inplace=True)



In [12]:
# Define revenue thresholds for the success categories
flop_threshold = 50000000  # $50 million
moderate_success_threshold = 100000000  # $100 million
hit_threshold = 200000000  # $200 million

In [16]:
# Define "Box Office Success" categories based on revenue thresholds
def categorize_success(revenue):
    if revenue < flop_threshold:
        return "Flop"
    elif revenue < moderate_success_threshold:
        return "Moderate Success"
    elif revenue < hit_threshold:
        return "Hit"
    else:
        return "Blockbuster"

df['BoxOfficeSuccess'] = df['revenue'].apply(categorize_success)


# Combine features and target variable
features = pd.concat([genres_dummies, keywords_dummies, prod_companies_dummies, prod_countries_dummies, spoken_lang_dummies], axis=1)
features = pd.concat([features, df[['budget', 'vote_count', 'vote_average']]], axis=1)
target = df['BoxOfficeSuccess']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variable
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Initialize and train the XGBoost model
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train_encoded)

# Predict on the test set
y_pred_xgb_encoded = xgb_model.predict(X_test)

# Inverse transform to get original class labels
y_pred_xgb = label_encoder.inverse_transform(y_pred_xgb_encoded)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy (XGBoost): {accuracy_xgb:.2f}")
print(classification_report(y_test, y_pred_xgb))


Accuracy (XGBoost): 0.76
                  precision    recall  f1-score   support

     Blockbuster       0.77      0.72      0.74       119
            Flop       0.86      0.94      0.89       606
             Hit       0.47      0.48      0.47       112
Moderate Success       0.39      0.22      0.28       124

        accuracy                           0.76       961
       macro avg       0.62      0.59      0.60       961
    weighted avg       0.74      0.76      0.75       961

